## interactive climate map in Bokeh (& Geopandas) and temperature forecast from fbprophet
    

- **CREDITS**:
    - To Paul Wlodkowski for ideas and data clean-up he did for our SPICED lesson
    - The data for this particular lesson was scraped from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/) and cleaned / pre-processed ahead of time.

In [1]:
import pandas as pd
import geopandas as gpd

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import GeoJSONDataSource
from bokeh.palettes import brewer
from bokeh.models import LinearColorMapper
from bokeh.models import ColorBar, HoverTool
from bokeh.models import Slider
from bokeh.layouts import widgetbox, column
from bokeh.io import curdoc


### Load Data Sets.
- the 'all_years.csv' was created by combining the Berkeley data with the fbprophet predictions from 'Week5_Berkel_world_forecastingProphet_mk.ipynb'

In [2]:
DATA = '../bokeh/all_years.csv' 

In [3]:
df = pd.read_csv(DATA)

In [38]:
df.loc[df['country'] != 'Antarctica'] # Antarctica has to be removed due to missing values

,country,year,monthly_anomaly
0,Afghanistan,1900,-0.311500
1,Afghanistan,1901,-0.166833
2,Afghanistan,1902,0.445000
3,Afghanistan,1903,-1.074417
4,Afghanistan,1904,-0.255083
...,...,...,...
31581,Åland,2029,1.383797
31582,Åland,2030,1.403843
31583,Åland,2031,1.423889
31584,Åland,2032,1.443981


In [34]:
SHAPEFILE = '../data/ne_110m_admin_0_countries.shp'

In [35]:
gdf = gpd.read_file(SHAPEFILE)[['ADMIN', 'geometry']] # select columns 'geometry: coordinates of each country'

In [36]:
df = df.groupby(['country', 'year'])[['monthly_anomaly']].mean().reset_index() ## double [[]] for monthly_anomaly to get a pdDataFrame back and not a list

### Merge Data Sets.
- We want to have our temperature data and geometric data in one place.
- Make sure you're still left with a GeoDataFrame at the end. # put in the gdf as left for merge!!!

In [10]:
gdf_merged = pd.merge(left = gdf, right = df, left_on = 'ADMIN', right_on = 'country')

In [11]:
gdf_merged.tail()

,ADMIN,geometry,country,year,monthly_anomaly
22791,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,2029,0.959232
22792,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,2030,0.976811
22793,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,2031,0.994390
22794,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,2032,1.012009
22795,Trinidad and Tobago,"POLYGON ((-61.68000 10.76000, -61.10500 10.890...",Trinidad and Tobago,2033,1.026658


   ## 5a. Generate a blank canvas / figure. 
   ### this follows basically the tutorial for bokeh

### 5b. Generate a GeoJSON for a single year and use it to add shapes onto the figure
- Let's use the year 2000 as an example.
- **Programming Tip**: 
    - If we can write code to work for a single year (hardcoded), then we can generalize this later to work for *any*  year!

In [12]:
gdf_2000 = gdf_merged[gdf_merged['year'] == 2019]
json_2000 = gdf_2000.to_json()

In [13]:
gdf_2000.head()

,ADMIN,geometry,country,year,monthly_anomaly
119,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,2019,0.529469
253,United Republic of Tanzania,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",United Republic of Tanzania,2019,0.931828
387,Western Sahara,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",Western Sahara,2019,1.225730
521,Canada,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,2019,1.457110
655,United States of America,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",United States of America,2019,0.967292


In [14]:
gdf_merged.head()

,ADMIN,geometry,country,year,monthly_anomaly
0,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,1900,-0.779583
1,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,1901,-0.763000
2,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,1902,-1.068083
3,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,1903,-0.409333
4,Fiji,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,1904,-0.826417


In [40]:
print(str(gdf_merged['monthly_anomaly'].min())) # necessary to see the range of temperature to be set
print(str(gdf_merged['monthly_anomaly'].max()))

-2.3870833333333334
3.0295833333333335


In [16]:
def get_geojson(yr):
    """Input a year (int) and return corresponding GeoJSON"""
    gdf_year = gdf_merged[gdf_merged['year'] == yr] 
    return gdf_year.to_json()
geosource = GeoJSONDataSource(geojson = get_geojson(2019))

In [17]:
geosource = GeoJSONDataSource(geojson = json_2000)

In [18]:
hover = HoverTool(tooltips = [ ('Country','@country'), ('Temp. Anomaly', '@monthly_anomaly')])

In [19]:
slider = Slider(title = 'Year', start = 1900, end = 2032, step = 1, value = 2013)
#define the constraints of the year slider

In [20]:
p = figure(title = 'Avg. Monthly Temperature Anomaly for Year 1900',
           plot_height = 400,
           plot_width = 600,
           tools=[hover]
          )

In [21]:
p.tools.append(hover)

In [22]:
palette = brewer['RdBu'][11]

In [23]:
color_mapper = LinearColorMapper(palette = palette,
                                 low = -3.5,
                                 high = 3.5, 
                                 nan_color = 'cornflowerblue')

In [24]:
color_bar = ColorBar(color_mapper = color_mapper,
                     label_standoff = 8,
                     width =  500,
                     height = 20,
                     location = (0,0),
                     orientation = 'horizontal'
                    )

In [25]:
p.patches('xs',
          'ys',
          source = geosource,
          fill_color = {'field' :'monthly_anomaly', 'transform': color_mapper},
          line_color = 'blue',
          line_width = 1)

GlyphRenderer(id='1034', ...)

In [26]:
p.add_layout(color_bar, 'below')

In [27]:
show(p)

In [28]:
def update_plot(attr, old, new):
    
    """Change properties / attributes of the datasource and title depending on slider value / position."""
    
    yr = slider.value
    new_data = get_geojson(yr) #our custom function from before
    geosource.geojson = new_data
    p.title.text = f'Avg. Monthly Temperature Anomaly for Year {yr}'
      

In [29]:
slider.on_change('value', update_plot)

In [30]:
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

RuntimeError: Models must be owned by only a single document, LinearScale(id='1012', ...) is already in a doc

**To view this application in interactive mode you need to set up a local Bokeh server.**

**In the terminal, run:**

``bokeh serve --show <name_of_notebook>.ipynb`` \
``bokeh serve --show <name_of_script>.py``